<a href="https://colab.research.google.com/github/axelsavrov/ML-for-Medical-Students/blob/main/Day3_TUBS_UNAM_Style_Transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center>
<img src="https://www.plri.de/assets/images/logo_plri_de.png"
width="200">
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/9d/Siegel_TU_Braunschweig_transparent.svg/800px-Siegel_TU_Braunschweig_transparent.svg.png"
width="250">
<img src="https://upload.wikimedia.org/wikipedia/commons/3/3d/Medizinische_Hochschule_Hannover_logo.svg"
width="250">
<img src="https://images.seeklogo.com/logo-png/38/2/universidad-nacional-autonoma-de-mexico-unam-logo-png_seeklogo-387361.png" width="110">
</center>

<font color="Teal" face="Georgia,arial">
  <h1 align="center"><b></b></h1>
  <h1 align="center"><i><b>Style Transfer</b></i></h1>
  </font>
  <font color="Black" face="Georgia,arial">
  <h5 align="center"><i><b>Voluntarios de TUBS</b></i></h5>
  <h5 align="center"><i><b>mail: unam.tubs@gmail.com</b></i></h5>
</font>

**CONTENTS**

Neural Style Transfer: The main idea is to combine the content of a “real” image with the artistic style of another image.
This process is based on deep neural networks, which are capable of extracting the content of one image (shapes, structure) and the style of another (colors, textures, strokes), and merging them into a new image.

- Style_Transfer.ipynb: Main notebook where all the code is ready to be executed step by step.
- Image gallery.

----------------------------------------

**🚀 HOW TO USE THE CODE?**

1. Open the Style_Transfer.ipynb notebook in Google Colab.

* Go to the menu: File > Save a copy to Drive
* Gray cells with code are executed with Shift + Enter

----------------------------------------

**🔧 HOW TO RUN THE NOTEBOOK?**

1. Locate the block of code you want to run.
In the upper left corner of the block, click on the triangle button ▶️

2. Wait for a green check mark ✅ to appear next to the block: this indicates that it has finished executing.

----------------------------------------

**📝 ADDITIONAL NOTES**

- The code may take between 10 and 15 minutes to load.

----------------------------------------
**🙋 IF YOU HAVE ANY QUESTIONS ABOUT HOW TO USE THE CODE, YOU CAN SEND US AN EMAIL**

Email for questions -- unam.tubs@gmail.com

Attach a screenshot if you get an error message.



# Setup

__Import libraries used by thisnotebook.__

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt


import torchvision.transforms as transforms
import torchvision.models as models

import copy

__Select computation device accordingly to what is available.__

If this machine have a gpu (cuda is available) we use it. Otherwise our fallback is the CPU.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

__Hyperparameters__

Hyperparameters control the number of iteration, size of image, and relative ration between losses (content, style, and denoising).

Fine tuning this parameter will affect the final result of the picture.

In [ ]:
ITERATIONS = 20 # Number of iterations
IMAGE_WIDTH = 512 # 512x512 is fine for HD images
IMAGE_HEIGHT = 512
CONTENT_WEIGHT = 1 # Weight of the content loss
STYLE_WEIGHT = 0.01 # Weight og the style loss
TOTAL_VARIATION_WEIGHT = 15 # Weight of the total variation loss
INPUT_IMAGE = 'noise' # Can be : 'noise', 'content', 'style', or a path to a file.

Thw next parameters are the heart of the algorithm, They deeply affect the result of combining style and content, because they define what IS the style and content.

To compute style and content, we use a deep neural network called VGG. Here is how VGG looks like inside.
![VGG Layers](https://www.researchgate.net/profile/Clifford_Yang/publication/325137356/figure/fig2/AS:670371271413777@1536840374533/llustration-of-the-network-architecture-of-VGG-19-model-conv-means-convolution-FC-means.jpg)

Tu compute the style of an image, we run the image through VGG. Then, we look at what values flowed across the layers (named _conv1_1_, _conv1_2_, etc.) and keep some of them.

For the __content__, we usually keep the first layers. The last convolutional layer from the second block is always a good bet.

For the __style__, a mix of the first conv layers (they contain color and texture informations) and last layers (they contain complexe features like trees, shapes, eyes, etc...) gives the best results.

---


__The names of layers you can add the the following lists in order to define the style and content loss are:__
  * `conv_n` where `n` is the index of the convolutional layer from the input.
  * `relu_n` where `n` is the index of the rectified linear layer from the input.
  * `pool_n` where `n` is the index of the max pooling layer from the input.
  * `bn_n` zhere `n` is the index of the batch normalization layer from the input.
  
  
  For example, convolutional layers are indexed from `conv_1` to `conv_16`.

In [ ]:
content_layers = ['conv_1', 'conv_2', 'conv_4']
style_layers = ['conv_2', 'conv_3', 'conv_4', 'conv_7', 'conv_10', 'conv_8']

Names under which the images will be saved.

In [ ]:
content_image_path = "content.png"
style_image_path = "style.png"
output_image_path = "result.png"
combined_image_path = "combined.png"

# Image selection

### Images

We download images from the web, but you can also specify the path to an image stored on your computer.

How do you do this?
Just drag and drop your file into the menu on the left, under the “Files” tab, and then type the file name (including the extension) instead of the URL.

Note:
 If you want to upload your own images, here you can see step by step how to do it. The link will be in the part of the code where it says “#my_content_image.” Just uncomment that line (by removing the # symbol) and write the path to your image.





In [ ]:
# Content images
san_francisco = "https://www.economist.com/sites/default/files/images/print-edition/20180602_USP001_0.jpg"
cat = "https://media.istockphoto.com/id/1443562748/es/foto/lindo-gato-de-jengibre.jpg?s=612x612&w=0&k=20&c=JVC5Z3LxpaTQaXu_fMZjIb73r39z6b0SnAxvNI8iZG0="
dog = "https://raw.githubusercontent.com/gaganbahga/cosine_art/master/test_images/test_image_2.jpg"
doge = "https://raw.githubusercontent.com/codazzo/dogr/master/public/doge.png"
#my_content_image = "/content/imagen_contenido.jpg"

# Style images:
tytus = "http://meetingbenches.com/wp-content/flagallery/tytus-brzozowski-polish-architect-and-watercolorist-a-fairy-tale-in-warsaw/tytus_brzozowski_13.jpg" #Warsaw by Tytus Brzozowski, http://t-b.pl
picasso_blue = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQAvqs_gleJBIg3673vZDnuKS-QsGA60JlniA&s"
elephant = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQyz-jrx9Na9NZg0oBAx50m7ahJ5VO2yUSvKCCexiNzi0z98yfp"
colorswir = "https://paintings.pinotspalette.com/colorful-swirly-sky-tv.jpg?v=10026315"
scream = "https://painting-planet.com/images/1/image046.jpg"
#my_style_image = "/content/imagen_estilo.jpg"

#my_style_image =


# The function wich allow us to download / open an image
def open_image(url):
  if url.startswith("http"):
    return Image.open(BytesIO(requests.get(url).content))
  else:
    return Image.open(url)


This is where you select your __Content image__ and __Style Image__.

It will be more fun if you use your own images.

In [ ]:
content_url = my_content_image
style_url = my_style_image

### Image loading

In [ ]:
#@title This is the _content_ image
content_image = open_image(content_url)
content_image = content_image.resize((IMAGE_WIDTH, IMAGE_HEIGHT))
content_image.save(content_image_path)
content_image

In [ ]:
#@title This is the _style_ image
style_image = open_image(style_url)
style_image = style_image.resize((IMAGE_WIDTH, IMAGE_HEIGHT))
style_image.save(style_image_path)
style_image

# Build the algorithm

## Feature loss and total variation

### Feature losses

The feature loss is a loss based on which features a neural network find inside an image. We have two such losses.


The first loss is the content loss, based on what content an image contain.
It is basically like a Mean Square Error. But instead of computing the MSE on the content image's pixels and the computed image's pixels, we compute the MSE between the activation of some layers of VGG19 after running our images through it.

To compute the activation layer, we add a pytorch block named ContentLoss inside the VGG network.

In [ ]:
class ContentLoss(nn.Module):
  def __init__(self, target,):
    super(ContentLoss, self).__init__()
    # We 'detach' the target content from the tree used
    # to dynamically compute the gradient: this is a stated value,
    # not a variable.
    self.target = target.detach()

  def forward(self, input):
    self.loss = F.mse_loss(input, self.target)
    return input

The second loss is the style loss. Instead of looking directly to activation of the layer, we first make a computation that compute somehow  the _angle_ between the features detected by the neural network.

We compute the [gram matrix](https://en.wikipedia.org/wiki/Gramian_matrix), which is a matrix whose coefficients are the dot product of two features vectores in the same activation layer from a single image.

We then have for each activation layer, two gram matrix (one from the style image, and one from the cotent image). We simply compute the MSE between the pairs of matrices.

The following code allow to plug a layer in VGG19 retreiving the gram matrices from each activation layer of interest.

---
__Why do I call this values _angles_?__

The Gram matrix is the matrix $G_{ij} = <v_i, v_j>$ is obtained by doing the dot product of each pair of feature vector. The dot product of two vectors $u_i$ and $v_i$ in $\mathbb{R}^3$ can be expressed as $||u_i|| .  ||v_i|| . \cos(u_i, v_i)$ where the two first terms are the length of the vectors, and the third is the cosine of the angle between this vectors.
Therefor, you can see the value of this dot product as a function of the _angle between two features_.

In [ ]:
class StyleLoss(nn.Module):
  def __init__(self, target_feature):
    super(StyleLoss, self).__init__()
    self.target = self.gram_matrix(target_feature).detach()

  def forward(self, input):
    G = self.gram_matrix(input)
    self.loss = F.mse_loss(G, self.target)
    return input

  @staticmethod
  def gram_matrix(input):
    a, b, c, d = input.size()
    # Here:
    # a is the batch size(=1)
    # b is the number of feature maps
    # (c,d) are the dimensions of a feature map

    # We reshape the activation layer into a collection of feature vectors
    features = input.view(a * b, c * d)

    # Compute the gram product
    G = torch.mm(features, features.t())

    # We 'normalize' the values of the gram matrix
    # by dividing by the norm of gram matrix filled with ones.
    return G.div((a * b * c * d) ** 0.5)

### Total Variation

The last loss is a _smoothing loss_.



This is called the total variation because it basically count the amplitude of all the variations happening in the image. Hight amplitude variation are caused by high constrats / brutal change of pixel values.

Minimizing the total variation of an image can to reducing noise (it is called [total variation denoising](https://en.wikipedia.org/wiki/Total_variation_denoising)). But you have to be carefull, because an image with a minimal total variation is a uniformly colored square. And it would be a really boring and uninteresting picture.

Too much total variation weight will lead to darker / less colorful images. But too few of it will generate very noisy images, without the "brush painting" style we are often loooking for in style transfer.

Since this is computed only from the resulting image , we just need a function to compute it.

---
_How is computed total variation ?_

Total variation is express by $\sum_{i, j} \sqrt{(x_{i+1, j}-x_{i, j})^2 + (x_{i+1, j}-x_{i, j})^2}$ where $x_{i, j}$ is the pixel at coordinates $(i, j)$. This is basically the euclidean norm of all the partial horizontal and vertical derivative of the surface given by $i, j \mapsto x_{i, j}$.
We are basically summing all the local derivative along the height map defined by the image.

This formula have the nice property to give a value which is invariant (up to an error introduced by the discrete nature of the image) by rotation.


In [ ]:
def total_variation_loss(x): # Expect a mini batch of dim NxWxH
  image_height = x.shape[1]
  image_width = x.shape[2]
  dx = x[:, :image_height-1, :image_width-1, :] - x[:, 1:, :image_width-1, :]
  dy = x[:, :image_height-1, :image_width-1, :] - x[:, :image_height-1, 1:, :]
  loss = (dx ** 2 + dy ** 2).sum() ** 0.5
  # Return loss normalized by image and batch size
  return loss / (image_width * image_height * x.shape[0])

### Normalization

We add one more block to automatically rescale and remove the mean of the images going through VGG19.

While training neural network on image dataset, all imges are their mean removed, and values are rescaled according to the standard deviation of the dataset. This is a common processing which makes the neural network convergence easier. The only implication is that once we want to use this network, we have to normalise the input image according to the coefficients used during training.

In [ ]:
# Create a module to normalize input image so we can easily put it in a
# nn.Sequential
class Normalization(nn.Module):
  def __init__(self, mean, std):
    super(Normalization, self).__init__()
    # Reshape the mean and std to make them [C x 1 x 1] so that they can
    # directly broadcast to image Tensor of shape [B x C x H x W].
    # B is batch size. C is number of channels. H is height and W is width.
    self.mean = torch.tensor(mean).view(-1, 1, 1)
    self.std = torch.tensor(std).view(-1, 1, 1)

  def forward(self, img):
    # Normalize img
    return (img - self.mean) / self.std


# Download and build VGG19

We now load our pretrained neural network (VGG19) for computing the losses.

In [ ]:
# Neural network used.
cnn = models.vgg19(pretrained=True).features.to(device).eval()

# Normalization mean and standard deviation.
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

We now build the actual network which will be called hundreads of times by the style transfer algorithm.

We only keep the layers required to compute the feature loss, and throw everything else away. We interleave our Normalization, StyleLoss and ContentLoss layers between VGG19's layers.

In [ ]:
def get_model_and_losses(style_img, content_img,
                               cnn=cnn,
                               cnn_normalization_mean=cnn_normalization_mean,
                               cnn_normalization_std=cnn_normalization_std):
  # We make a deep copy of vgg19 in order to not modify the original
  cnn = copy.deepcopy(cnn)

  # Normalization module
  normalization = Normalization(cnn_normalization_mean, cnn_normalization_std).to(device)

  # This list will contain the losses computed by the network.
  content_losses = []
  style_losses = []

  # We rebuild the model as a nn sequential whose first layer is
  # our normalization layer.
  model = nn.Sequential(normalization)

  # We brows the layer of `cnn` and stack them into `model`.
  i = 0  # Incremented every time we see a conv layer.
  for layer in cnn.children():

    if isinstance(layer, nn.Conv2d):
        i += 1
        name = 'conv_{}'.format(i)
    elif isinstance(layer, nn.ReLU):
        name = 'relu_{}'.format(i)
        # The in-place version doesn't play very nicely with the ContentLoss
        # and StyleLoss we insert below. So we replace with out-of-place
        # ones here.
        layer = nn.ReLU(inplace=False)
    elif isinstance(layer, nn.MaxPool2d):
        name = 'pool_{}'.format(i)
    elif isinstance(layer, nn.BatchNorm2d):
        name = 'bn_{}'.format(i)
    else:
        raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

    model.add_module(name, layer)

    # Check if the layer we just added was in the content layer list.
    # If so, we just stack a Content Loss layer.
    if name in content_layers:
      target = model(content_img).detach()
      content_loss = ContentLoss(target)
      model.add_module("content_loss_{}".format(i), content_loss)
      content_losses.append(content_loss)

    # Check if the layer we just added was in the style layer list.
    # If so, we just stack a Style Loss layer.
    if name in style_layers:
      target_feature = model(style_img).detach()
      style_loss = StyleLoss(target_feature)
      model.add_module("style_loss_{}".format(i), style_loss)
      style_losses.append(style_loss)

  # Now we trim off the layers after the last content and style losses
  # to keep the model as small as possible.
  for i in range(len(model) - 1, -1, -1):
      if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
          break

  model = model[:(i + 1)]

  return model, style_losses, content_losses

# Running the algorithm

We write a function converting PIL images to a `torch.Tensor` and apply it to our input images.

In [ ]:
def image_to_tensor(image):
  # Transform to tensor
  image = transforms.ToTensor()(image)
  # Fake batch dimension required to fit network's input dimensions
  image = image.unsqueeze(0)
  # Move to the right device and convert to float
  return image.to(device, torch.float)

content_img = image_to_tensor(content_image)
style_img = image_to_tensor(style_image)

We now declare the inverse function, converting back tensor to images.

In [ ]:
# Reconvert a tensor into PIL image
def tensor_to_image(tensor):
  img = (255 * tensor).cpu().detach().squeeze(0).numpy()
  img = img.clip(0, 255).transpose(1, 2, 0).astype("uint8")
  return Image.fromarray(img)

We now generate or load the input image from which we are going to converge.

In [ ]:
#@title This is the _input_ image
if INPUT_IMAGE == 'noise':
  input_img = torch.randn(content_img.data.size(), device=device)
elif INPUT_IMAGE == 'content':
  input_img = content_img.clone()
elif INPUT_IMAGE == 'style':
  input_img = style_img.clone()
else:
  image = open_image(INPUT_IMAGE).resize((IMAGE_SIZE, IMAGE_SIZE))
  input_img = image_to_tensor(image)
  input_img += torch.randn(content_img.data.size(), device=device)*0.1

# To visualise it better, instead of clipping values, we rescale
# them to fit [-1,1], and convert to an image. This is mostly because
# the visualization given is closer to what the actual values stored
# in the tensor are.
if INPUT_IMAGE == 'noise':
  img = transforms.ToPILImage()(input_img[0].cpu())
else:
  img = tensor_to_image(input_img[0])
img

We have to choose an optimisation algorithm for our gradient descent.
The algorithm giving the best visual result is LBFGS. But you can try other algorithms like Adam, RAdam, SGD...

In [ ]:
def get_input_optimizer(input_img):
  # This line tell LBFGS what parameters we should optimize
  optimizer = optim.LBFGS([input_img.requires_grad_()])
  #optimizer = optim.Adam([input_img.requires_grad_()])
  return optimizer

We add one more function in order to display the evolution of the input image while the algorithm run.

In [ ]:
def show_evolution(tensor, history=[], title=None):
    image = tensor.cpu().clone().squeeze(0)
    image = tensor_to_image(image)
    # Display a big figure
    plt.rcParams['figure.figsize'] = [10, 5]
    plt.figure()
    # Image
    plt.subplot(121)
    plt.imshow(image)
    plt.axis('off')
    if title is not None:
        plt.title(title)
    # Losses
    ax = plt.subplot(122)
    plt.yscale('log')
    plt.title('Losses')
    import numpy as np
    history = np.array(history).T
    plt.plot(history[0], label='Style')
    plt.plot(history[1], label='Content')
    plt.plot(history[2], label='Variation')
    plt.plot(history[3], label='Sum')
    plt.legend(loc="upper right")
    # Finaly show the graph
    plt.show()
    # Display a textual message
    print('Style Loss : {:4f} Content Loss: {:4f} Variation Loss: {:4f} Sum: {:4f}'.format(
        history[0][-1], history[1][-1], history[2][-1], history[3][-1]))

We are now ready to converge toward our AI painted image.

In [ ]:
print('Building the style transfer model..')
model, style_losses, content_losses = get_model_and_losses(style_img, content_img)
optimizer = get_input_optimizer(input_img)

print('Optimizing..')
iterations = 0
history=[]
while iterations <= ITERATIONS * 100:
  # Compute the loss and backpropagate to the input_image.
  # (The LBFGS optimizer only accept work through closures.)
  def closure():
      global history
      global iterations

      optimizer.zero_grad()

      # Compute the total variation loss
      variation_score = total_variation_loss(input_img) * TOTAL_VARIATION_WEIGHT
      # Compute the features through the model
      model(input_img)
      # Compute style and content losses
      style_score = sum(sl.loss for sl in style_losses)
      style_score *= STYLE_WEIGHT / len(style_losses)
      content_score = sum(cl.loss for cl in content_losses)
      content_score *= CONTENT_WEIGHT / len(content_losses)
      # Our global loss is the sum of the 3 values
      loss = style_score + content_score + variation_score
      # Save the value of loss in order to draw them as a graph
      history += [[style_score.item(), content_score.item(), variation_score.item(), loss.item()]]

      # If the iteration is a multiple of 100, display some informations
      iterations += 1
      if iterations % 100 == 0:
          show_evolution(input_img.data.clone().detach().clamp(0, 1), history, title="Iteration %d:" % iterations)

      # Backpropagate gradients and leave the optimizer do his job.
      loss.backward()
      return loss

  optimizer.step(closure)


In [ ]:
#@title Our beautiful result
img = tensor_to_image(input_img)
img.save(output_image_path)
img

In [ ]:
#@title Visualise combined results
combined = Image.new("RGB", (IMAGE_WIDTH*3, IMAGE_HEIGHT))
x_offset = 0
for image in map(Image.open, [content_image_path, style_image_path, output_image_path]):
    combined.paste(image, (x_offset, 0))
    x_offset += IMAGE_WIDTH
combined.save(combined_image_path)
combined

In [ ]:
import torch

# Crear un tensor de 2 dimensiones (matriz)
tensor_2d = torch.tensor([[1, 2, 3], [4, 5, 6]])
print("Tensor de 2 dimensiones:")
print(tensor_2d)
print("")

# Crear un tensor de 3 dimensiones
tensor_3d = torch.tensor([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])
print("Tensor de 3 dimensiones:")
print(tensor_3d)
print("")

# Crear un tensor de 4 dimensiones
tensor_4d = torch.tensor([[[[1, 2], [3, 4]], [[5, 6], [7, 8]]], [[[9, 10], [11, 12]], [[13, 14], [15, 16]]]])
print("Tensor de 4 dimensiones:")
print(tensor_4d)


#**We want to see your creation!**

##Share the results of your exercise by uploading your image at the following link. We would love to see how you combined content and style in your own unique way.

##👉 [Upload your image here](https://docs.google.com/forms/d/e/1FAIpQLSdawwLBC8Hmy0D-djRLu4uS-CwnyvOEr8ir4ovw9iBRgfvQ4w/viewform?usp=dialog)

**Remember: each result is unique, and we can all learn from seeing each other's work. Go ahead and share it!**
